In [1]:
import numpy as np

In [2]:
from sklearn.metrics import mean_squared_error

In [3]:
from __future__ import division

In [4]:
train_data = np.loadtxt('train.txt')

In [80]:
train_data[:10]

array([[ 1.,  1.,  5.],
       [ 1.,  2.,  3.],
       [ 1.,  3.,  4.],
       [ 1.,  4.,  3.],
       [ 1.,  5.,  3.],
       [ 1.,  6.,  5.],
       [ 1.,  7.,  4.],
       [ 1.,  8.,  1.],
       [ 1.,  9.,  5.],
       [ 1., 10.,  3.]])

In [5]:
from multiprocessing.dummy import Pool as ThreadPool


In [6]:
pool = ThreadPool(10)

In [62]:
class Factorization_Machines2:
    def __init__(self, max_epoch, alpha, rank, batch_size):
        self.max_epoch = max_epoch
        self.alpha = alpha
        self.rank = rank
        self.batch_size = batch_size
        self.errors = []
    
    def create_features(self, data):
        ### features one hot: film_id, user_id
        film_id = np.unique(data[:, 0])
        len_film_id = len(film_id)
        user_id = np.unique(data[:, 1])
        self.size_films = len(film_id)
        self.size_users = len(user_id)
        self.size_features = len(film_id) + len(user_id)
        features = np.zeros((len(data), self.size_features))
        for i in range(len(data)):
            features[i][int(data[i][0] - 1)] = 1
            features[i][len_film_id + np.where(user_id == data[i][1])[0][0]] = 1
        self.features = features
        
    def get_statistic(self, data, target):
        film_id, counts = np.unique(data[:, 0], return_counts = True)
        start = 0
        self.mean_film_mark = []
        for count in counts:
            self.mean_film_mark.append(np.mean(target[start : start + count]))
            start += count
        
        
        user_id, counts = np.unique(data[:, 1], return_counts = True)
        index = data[:, 1].argsort(axis = 0)
        sort_target = target[index]
        start = 0
        self.mean_user_mark = []
        for count in counts:
            self.mean_user_mark.append(np.mean(sort_target[start : start + count]))
            start += count   
    
    def grad_by_w0(self):
        return 1
    
    def grad_by_w(self, x):
        return x
    
    def grad_by_v(self, x, film_bool, user_bool):
        grad_v = np.zeros((self.size_features, self.rank))
        index = np.where(x == 1)[0]
        grad_v[index[0]] += self.v[index[1]] * film_bool
        grad_v[index[1]] += self.v[index[0]] * user_bool
       # print grad_v[index[0]], grad_v[index[1]]    
        return grad_v 
    
    def get_y(self, x):
        temp_sum = 0
        index = np.where(x == 1)[0]
        temp_sum += np.sum((self.v[index[0]] + self.v[index[1]])**2) - \
            np.sum(self.v[index[0]]**2) - np.sum(self.v[index[1]]**2)
        return self.w[0] + np.sum(self.w[1:] * x) + 0.5 * temp_sum
    
    def get_all_y(self, data):
        return np.array(pool.map(self.get_y, data))  
    
    def w_init(self):
        for i in range(self.size_films):
            self.w[i+1] = self.mean_film_mark[i] 
            self.w_statistic[i] = self.mean_film_mark[i]
        for i in range(self.size_users):
            self.w[self.size_films + i + 1] = self.mean_user_mark[i] 
            self.w_statistic[self.size_films + i] = self.mean_user_mark[i]
            
            
    def fast_grad_by_v_film(self, x, y):
        index = np.where(x == 1)[0]
        for i in range(self.rank):
            temp = 2*(self.get_y(x) - y)
            self.v[index[0]][i] -= self.alpha * temp * self.v[index[1]][i] 
        
    def fast_grad_by_v_user(self, x, y):
        index = np.where(x == 1)[0]
        for i in range(self.rank):
            temp = 2*(self.get_y(x) - y)
            self.v[index[1]][i] -= self.alpha * temp * self.v[index[0]][i]         
    
    def fit(self, data, target):
        N = len(target)
        self.create_features(data)
        self.get_statistic(data, target)
        self.w = np.zeros(self.size_features + 1)
        self.w_statistic = np.zeros(self.size_features + 1)
       # self.v = np.zeros((self.size_features, self.rank)) + 0.1
        self.w_init()
        self.v = np.random.normal(0, 0.1, (self.size_features, self.rank))
        film_bool = 0
        user_bool = 1
        for epoch in range(self.max_epoch):
            print "epoch: ", epoch
            index = np.arange(N)
            np.random.shuffle(index)
            for i in range(0, N, self.batch_size):
                grad_w0 = 0
                grad_w = 0
                grad_v = 0
                for j in index[i:i+self.batch_size]:
                    temp = 2*(self.get_y(self.features[j]) - target[j])
                    grad_w0 = grad_w0 + temp
                    grad_w = grad_w + temp * self.grad_by_w(self.features[j])
                    grad_v = grad_v + temp * self.grad_by_v(self.features[j], film_bool, user_bool)
                self.w[0] -= self.alpha * grad_w0 / self.batch_size 
                self.w[1:] -= self.alpha * grad_w / self.batch_size
               # print self.v[index[0]]
                self.v -= self.alpha * grad_v 
               # print self.v[index[0]]
                film_bool = (film_bool + 1) % 2  
                user_bool = (user_bool + 1) % 2 
                grad_w0 = 0
                grad_w = 0
                grad_v = 0
                for j in index[i:i+self.batch_size]:
                    temp = 2*(self.get_y(self.features[j]) - target[j])
                    grad_w0 = grad_w0 + temp
                    grad_w = grad_w + temp * self.grad_by_w(self.features[j])
                    grad_v = grad_v + temp * self.grad_by_v(self.features[j], film_bool, user_bool)
                self.w[0] -= self.alpha * grad_w0 / self.batch_size 
                self.w[1:] -= self.alpha * grad_w / self.batch_size
                
                self.v -= self.alpha * grad_v / self.batch_size

                self.errors.append(mean_squared_error(target, self.get_all_y(self.features)))   
                print "error: ", self.errors[-1]
                film_bool = (film_bool + 1) % 2  
                user_bool = (user_bool + 1) % 2             
        
        
    def create_features_for_newdata(self, data, new_data):
        ### features one hot: film_id, user_id
        film_id = np.unique(data[:, 0])
        user_id = np.unique(data[:, 1])
        features = np.zeros((len(new_data), self.size_features))
        for i in range(len(new_data)):
            film_id_index = np.where(film_id == new_data[i][0])[0]
            if len(film_id_index) != 0:
                features[i][film_id_index[0]] = 1
            user_id_index = np.where(user_id == new_data[i][1])[0]
            if len(user_id_index) != 0:
                features[i][self.size_films + user_id_index[0]] = 1
                
        return features
    
    def predict(self, new_data, data):
        features = self.create_features_for_newdata(data, new_data)
        pred = []
        for i, x in enumerate(features):
            if np.sum(x) == 2:
                #good
                pred.append(self.get_y(features[i]))
            if np.sum(x) == 1:
                #not good, not bad
                pred.append(self.w_statistic[np.where(x == 1)[0][0]])
            if np.sum(x) == 0:
                #ACHTUNG!
                pred.append(3)

        return pred
    
    

In [22]:
FM2 = Factorization_Machines2(100, 0.1, 20, 100)

In [23]:
FM2.fit(train_data[:, :2], train_data[:, 2])

epoch:  0
error:  6.219770849035344
error:  3.0357596732384455
error:  1.6564406944894383
error:  1.240101722244618
error:  1.0087489362350426
error:  0.9409172558254384
error:  0.8942648875535495
error:  0.8815532388193377
error:  0.8799907725066666
error:  0.8781131725716178
error:  0.880298441845358
error:  0.8829489743735387
error:  0.8817741706737218
error:  0.8804616572677404
error:  0.8785072464320524
error:  0.8786317561410703
error:  0.8778090857243119
error:  0.8781248017294789
error:  0.8787570512388237
error:  0.877506721470167
error:  0.8773796345928356
error:  0.8787972827125085
error:  0.877256519245263
error:  0.8772693105474072
error:  0.8771864446211995
error:  0.8771365209057603
error:  0.8791374447386742
error:  0.8768474959340925
error:  0.87971307213165
error:  0.8892785974627048
error:  0.8941155308708469
error:  0.8811425366109654
error:  0.8791977962197562
error:  0.8803259132878126
error:  0.883931148603348
error:  0.8848779404094685
error:  0.8844844121618982

error:  0.8660408752018736
error:  0.8628204600073691
error:  0.8632403510578234
error:  0.8627976489769085
error:  0.8626960776304622
error:  0.8646590166086731
error:  0.8653548789297947
error:  0.8641616419215311
error:  0.8636756084113666
error:  0.8626810325196522
error:  0.8624462309908982
error:  0.8624502319962508
error:  0.8624475743416324
error:  0.862978354026124
error:  0.8624770183562991
error:  0.8631400729492689
error:  0.8627296969184473
error:  0.8627178412716452
error:  0.8631558812253552
error:  0.8623817666417894
error:  0.8624975774651901
error:  0.8649824412326911
error:  0.8761162146728683
error:  0.8642941652003316
error:  0.8621686358534596
error:  0.8621505257724242
error:  0.8623273723804274
error:  0.8621772835643285
error:  0.8620652623475239
error:  0.8622773815656549
error:  0.8622908963757249
error:  0.8625636599319593
error:  0.8627380618712818
error:  0.8662814704023051
error:  0.8687798106457882
error:  0.8672467363917825
error:  0.8624773536906284
er

error:  0.8581607645206711
error:  0.8543546387352845
error:  0.8583857429227595
error:  0.8543743472425073
error:  0.8542416179264637
error:  0.8542322739965258
error:  0.8542349993803704
error:  0.854565209013968
error:  0.8551624126280697
error:  0.8553923174164626
error:  0.8542489386362377
error:  0.8541156238743693
error:  0.856457919876823
error:  0.8587280535914068
error:  0.8546588779871879
error:  0.8548053285699572
error:  0.8585124023498736
error:  0.854765387854204
error:  0.8540472602067607
error:  0.8545173714310395
error:  0.8541489136012073
error:  0.8562641523296041
error:  0.8549103005138372
error:  0.8571081299328042
error:  0.854402968793874
error:  0.8568155631056387
error:  0.85423355945756
error:  0.8538603911762938
error:  0.8550810441863921
error:  0.8540810307311566
error:  0.8537903216848738
error:  0.853809592699219
error:  0.8561307218961929
error:  0.8567628945153546
error:  0.8595909466857741
error:  0.8537720555914552
error:  0.8539191836815297
error:  

error:  0.8490237282204043
error:  0.8496331174554476
error:  0.8502780115662214
error:  0.8507762960042542
error:  0.849002850244953
error:  0.8492129080413002
error:  0.84899196002098
error:  0.8490097003247183
error:  0.849116064970368
error:  0.8489858358254146
error:  0.8489742022809496
error:  0.8506117971755556
error:  0.849349939785084
error:  0.8494537996073278
error:  0.8489644552674975
error:  0.8494296225693856
error:  0.8492930210864265
error:  0.8493508599574924
error:  0.8507888798796515
error:  0.8488815867712229
error:  0.8512629150230591
error:  0.8501319491331661
error:  0.8502860044898924
error:  0.8491760291470211
error:  0.8491390219164356
error:  0.8490672837571926
error:  0.8488478381233712
error:  0.849584916642941
error:  0.8496893390090753
error:  0.85158783900429
error:  0.8493944295361522
error:  0.8496408751196955
error:  0.8488471328202205
error:  0.8499123838831562
error:  0.8490264048891399
error:  0.8567672279929414
error:  0.8551815011843396
error:  0

error:  0.8465591259123345
error:  0.8465331205373021
error:  0.8468900945957962
error:  0.8466908297400343
error:  0.8504619549599584
error:  0.8466938453670183
error:  0.8476667885409569
error:  0.8468882443015117
error:  0.8471467177675017
error:  0.852969364768483
error:  0.8511032540940805
error:  0.8471581135612555
error:  0.8478080302211313
error:  0.8463767222480384
error:  0.84709846759971
error:  0.8463787976515004
error:  0.8473352605529839
error:  0.8509309050744346
error:  0.8468712276104605
error:  0.8463837805521668
error:  0.8489101938125954
error:  0.8463775129591755
error:  0.8473071078791994
error:  0.8463873403563108
error:  0.8502178151633706
error:  0.8483144787949971
error:  0.846808534277114
error:  0.8471789053210075
error:  0.8476995051691303
error:  0.8468458987981466
error:  0.8481576573126276
error:  0.8603349112413753
error:  0.8540176885991997
error:  0.8583151281484932
error:  0.8474700081486103
error:  0.8465092659953624
error:  0.8465659818798913
error

...

error:  0.824013409088116
error:  0.8279459357721142
error:  0.8276376076133174
error:  0.8235563445412503
error:  0.8268679809676308
error:  0.8242086491393891
error:  0.8235318240505565
error:  0.8240313483269682
error:  0.8239093392584979
error:  0.8239193072003343
error:  0.8239855769087968
error:  0.8235031461926893
error:  0.8235001577774939
error:  0.8246148595481919
error:  0.8235093059418905
error:  0.829620998670441
error:  0.82520114577357
error:  0.8235935150862098
error:  0.8238357322565997
error:  0.8234608584832382
error:  0.8237143366423045
error:  0.8256496781863553
error:  0.8243190880069132
error:  0.8247713694324381
error:  0.8247340318922786
error:  0.8238142565197765
error:  0.8236518844747314
error:  0.826703363408538
error:  0.8234298839376409
error:  0.823420632316505
error:  0.8235568953199396
error:  0.8234267059184677
error:  0.8277677143424603
error:  0.8255337250860341
error:  0.8261289718872662
error:  0.8239567981984427
error:  0.8234525246748898
error: 

error:  0.8243762776543273
error:  0.8226151575079726
error:  0.8230870293677439
error:  0.8260663028135647
error:  0.8229896698458796
error:  0.8226268947188029
error:  0.8227330719139967
error:  0.8270540469880777
error:  0.8230893866834232
error:  0.8227396307693184
error:  0.8231043504828963
error:  0.823290232139459
error:  0.8227899410330615
error:  0.8253634494861072
error:  0.8228754011855388
error:  0.8226292989156575
error:  0.828564479433579
error:  0.827074105531768
error:  0.8226362687721384
error:  0.8225815797237689
error:  0.8234545121587917
error:  0.82781834951534
error:  0.8273220630861762
error:  0.8255703809750877
error:  0.8236916303350583
error:  0.826125358217331
error:  0.8261313852067635
error:  0.8239823161591597
error:  0.8248865778053334
error:  0.8298181932161485
error:  0.8244619242542973
error:  0.8285303421079823
error:  0.8253748485412624
error:  0.8335491502127219
error:  0.8262962448303838
error:  0.8231955360248815
error:  0.8225011719815265
error: 

error:  0.8236563235359541
error:  0.8220880127712852
error:  0.822296856554712
error:  0.8223082900278481
error:  0.822201127684028
error:  0.8243823165773932
error:  0.8224303870140152
error:  0.8233097580345317
error:  0.8224313252403843
error:  0.8222781250172948
error:  0.8237157381463301
error:  0.8248453290626392
error:  0.8236125463835602
error:  0.8221053314872634
error:  0.8224591100596799
error:  0.8221401240344386
error:  0.8225085253690042
error:  0.8284911586991349
error:  0.8235655142539731
error:  0.8232192068186008
error:  0.8232431871567734
error:  0.8225741698296627
error:  0.8222049669523569
error:  0.8230163198732029
error:  0.8244152053597414
error:  0.822209148520508
error:  0.8243451168334069
error:  0.8227661292214727
error:  0.8225002555115196
error:  0.8222141671881588
error:  0.8294782215956541
error:  0.8222132645034186
error:  0.8246855701845638
error:  0.825798094662148
error:  0.8222600178055449
error:  0.8222010598042947
error:  0.822209529644883
error:

error:  0.8250351502767291
error:  0.8232320515790095
error:  0.82235868440781
error:  0.823815110694535
error:  0.821896937782657
error:  0.8218674220390944
error:  0.8221154935327415
error:  0.8224721337188782
error:  0.8221229601149183
error:  0.8224521688487362
error:  0.8223719169120651
error:  0.8217756364034212
error:  0.8221561933183194
error:  0.8221346404285298
error:  0.8224851033538192
error:  0.8220730553416488
error:  0.8275523352735115
error:  0.8283573069398925
error:  0.8258937705932127
error:  0.8223841373629377
error:  0.82349736237512
error:  0.822573960188486
error:  0.8219578550921892
error:  0.8217285920443028
error:  0.8217309268569775
error:  0.8221340609727125
error:  0.8219143424076016
error:  0.8227973172981363
error:  0.8220273057392712
error:  0.8222802876718841
error:  0.8221063466310545
error:  0.8234850681942918
error:  0.8273444666625324
error:  0.823860519081706
error:  0.8224116352200235
error:  0.8225520934787285
error:  0.8234823874993885
error:  0

KeyboardInterrupt: 